In [6]:
%matplotlib qt

In [2]:
# import 
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy

import sys
sys.path.append(r"/Users/god/bison_not_seminar")

In [3]:
# import for read a file (fif)
from src.Plotter.OfflinePlots.MPLPrinter import MPLPrinter
from src.Plotter.OfflinePlots.MPLPrinter import DetectionMode
from src.Unicorn_Recorder.Dummies import RealData
import mne
import os
from src.Utils import Utils
from pathlib import Path

In [4]:
# Data import 
datadir = Path(r'/Users/god/bison_not_seminar/visual_stress_data')
inputdatapath = datadir / '20-4_blur4px_font20pt_sansserif_count9_no1.fif'
#printer = MPLPrinter(250)

raw = mne.io.read_raw_fif(inputdatapath, preload=True)
#print("Raw shape ", raw)
events = raw.info["events"]
events = [event["list"] for event in events]
#print("Events", events)
data = raw.get_data(RealData.UNICORN_ELECTRODES)
print(data.shape)

for e in events:
    print(e)
    
channels = 8
channeldata = data[:channels, :]
print(channeldata.shape)

Opening raw data file /Users/god/bison_not_seminar/visual_stress_data/20-4_blur4px_font20pt_sansserif_count9_no1.fif...
Isotrak not found
    Range : 0 ... 40440 =      0.000 ...   161.760 secs
Ready.
Reading 0 ... 40440  =      0.000 ...   161.760 secs...
(17, 40441)
[  0   0 241]
[   1    0 5261]
[    0     0 10281]
[    1     0 15301]
[    0     0 20321]
[    1     0 25341]
[    0     0 30361]
[    1     0 35391]
(8, 40441)


<ipython-input-4-5703fcdeda2d>:6: RuntimeWarning: This filename (/Users/god/bison_not_seminar/visual_stress_data/20-4_blur4px_font20pt_sansserif_count9_no1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(inputdatapath, preload=True)


In [5]:
#eeg_data = channeldata
ch_types = ['eeg'] * 8
ch_names = ['EEG{}'.format(i) for i in range(8)]
sfreq = 250
info = mne.create_info (ch_names = ch_names, sfreq = sfreq, ch_types = ch_types)
#raw = mne.io.RawArray (eeg_data, info)
raw = mne.io.read_raw_fif(inputdatapath, preload=True)
print(raw.info)
event_dict = {'focused':0, 'blurred': 1}


# print(events[:,2])|
print("Shape of events is {}".format(events))
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'],
                          first_samp=raw.first_samp)
fig.subplots_adjust(right=0.7)  # make room for legend

#raw.notch_filter(numpy.arange(50, 100, 50), notch_widths=5, filter_length='auto',
#                 phase='zero')
# its time to plot something!
#raw.plot_psd(average = False)
#raw.plot(n_channels=len(raw.ch_names))
raw = raw.pick(['eeg'])

Opening raw data file /Users/god/bison_not_seminar/visual_stress_data/20-4_blur4px_font20pt_sansserif_count9_no1.fif...
Isotrak not found
    Range : 0 ... 40440 =      0.000 ...   161.760 secs
Ready.
Reading 0 ... 40440  =      0.000 ...   161.760 secs...
<Info | 10 non-empty values
 bads: []
 ch_names: EEG0, EEG1, EEG2, EEG3, EEG4, EEG5, EEG6, EEG7, EEG8, EEG9, ...
 chs: 8 EEG, 9 MISC
 custom_ref_applied: False
 events: 8 items (list)
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 17
 projs: []
 sfreq: 250.0 Hz
>
Shape of events is [array([  0,   0, 241], dtype=int32), array([   1,    0, 5261], dtype=int32), array([    0,     0, 10281], dtype=int32), array([    1,     0, 15301], dtype=int32), array([    0,     0, 20321], dtype=int32), array([    1,     0, 25341], dtype=int32), array([    0,     0, 30361], dtype=int32), array([    1,     0, 35391], dtype=int32)]


<ipython-input-5-d7690e2ac5af>:7: RuntimeWarning: This filename (/Users/god/bison_not_seminar/visual_stress_data/20-4_blur4px_font20pt_sansserif_count9_no1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(inputdatapath, preload=True)


In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""

    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
# Bucketing data 
X_train = []
y_train = []
samplerate = 250

windows = []
for i in range(len(events)):
    # data
    e = events[i]
    start_index = e[2]
    if i == len(events)-1:
        end_index = data.shape[1]
    else:
        end_index = events[i+1][2]
    windows.append(end_index - start_index)

window_size = int(min(windows)/samplerate)*samplerate
print('Window size {}'.format(window_size))


# loop over events
for i in range(len(events)):
    # get the event
    e = events[i]
    
    # window markers
    start_index = e[2]
    end_index = start_index + window_size
    
    # shape: (8, 250)
    example_buckets = [[[] for _ in range(channels)] for _ in range(20)] 
    
    # loop through the channels and cut out the window for each, append the window to tmp
    for chaNo in range(len(channeldata)):
        window = channeldata[chaNo][start_index:end_index]
        chunked_windows = list(chunks(window, samplerate))
        # expecting len = 20
        for i in range(len(chunked_windows)):
            example_buckets[i][chaNo] = chunked_windows[i]
        #print(numpy.asarray(example_buckets).shape)
    
    chunks_per_window = len(example_buckets)
    for _ in range(chunks_per_window):
        y_train.append(e[0])
        
        
    for example in example_buckets:
        X_train.append(example)
    
X_train = numpy.asarray(X_train)
y_train = numpy.asarray(y_train)
print(X_train.shape)
print(y_train.shape)



In [ ]:
X_train = X_train.reshape(X_train.shape[0], -1)
print(X_train.shape)

In [ ]:
# Data processing and formating

In [ ]:
# model
model = Sequential()
model.add(keras.Input(shape=samplerate*channels))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
# compilation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fit model
training = model.fit(X_train, y_train, epochs=50, batch_size=16)

In [ ]:
# predictions
predictions = model.predict(X_test)